In [1]:
import pandas as pd
from matplotlib import pyplot as plt
from matplotlib.pyplot import figure
import numpy as np
import matplotlib as mpl
import seaborn as sns
from scipy.stats import ttest_ind
import re
%matplotlib inline

In [2]:
# dados TODOS os rois
df = pd.read_csv('data/KU80_INIFAK_SIFAK_DOX and DNAPK_DOX gh2ax_foci.csv')
# ----------------------------------------------------------------------------------------------------------------------------------
def coef_pc(data,nome_data):
    '''Calcula coeficiente de pearson das colunas id_first e id_second da data selecionada com o nome da data'''
    first_array = data['id_gh2ax'] 
    second_array = data['id_fak'] 
    pearson_first_second = np.corrcoef(first_array, second_array)[0][1]
    print("O coeficiente de pearson do: " + str(nome_data) + " é " + str(pearson_first_second))
    string_para_print = ("Correlação de pearson: \n" + str(pearson_first_second))
    return string_para_print
# ----------------------------------------------------------------------------------------------------------------------------------

In [3]:
df = df .drop(columns=['nucleos'])
df['image'] = df['label'].str.extract(r'(\d+)_Out_Channel')
df['nucleo'] = df['label'].str.extract(r'nucleo_(\d+)')

In [4]:
df.to_csv("all_rois_clean.csv", encoding=("utf-8-sig"))

In [5]:
df_2 = pd.read_csv("all_rois_clean.csv")

In [6]:
df_2 = df_2[df_2['prot'] != 'dnapk']
tto_list = ['dox', 'dox_dmso', 'dox_pf', 'dox_sc', 'dox_sifak']

pc_df_2 = pd.DataFrame()

for tto in tto_list:
    image_list = df_2[(df_2['tto'] == tto)]["image"].unique()
    for image in image_list:
        nucleos_list = df_2[(df_2['tto'] == tto) & (df_2['image'] == image)]['nucleo'].unique()
        for nucleo in nucleos_list:
            id_ku80 = df_2[(df_2['tto'] == tto) & (df_2['image'] == image) & (df_2['nucleo'] == nucleo)]['id_ku80']
            id_gh2ax = df_2[(df_2['tto'] == tto) & (df_2['image'] == image) & (df_2['nucleo'] == nucleo)]['id_gh2ax']
            id_fak = df_2[(df_2['tto'] == tto) & (df_2['image'] == image) & (df_2['nucleo'] == nucleo)]['id_fak']
            total_rois = df_2[(df_2['tto'] == tto) & (df_2['image'] == image) & (df_2['nucleo'] == nucleo)].shape[0]
        
            pearson_id_gh2ax_id_ku80 = np.corrcoef(id_gh2ax, id_ku80)[0][1]
            pearson_id_fak_id_ku80 = np.corrcoef(id_fak, id_ku80)[0][1]
            pearson_id_fak_id_gh2ax = np.corrcoef(id_fak, id_gh2ax)[0][1]
            
            df_2_rcorr = pd.DataFrame(np.transpose([[image],[tto], [nucleo], [total_rois], [pearson_id_gh2ax_id_ku80], [pearson_id_fak_id_ku80], [pearson_id_fak_id_gh2ax]]), 
                                    columns=['image','tto', 'nucleo', 'nrois', 'pc_gh2ax_ku80', 'pc_fak_ku80', 'pc_gh2ax_fak'])
            pc_df_2 = pd.concat([pc_df_2, df_2_rcorr], ignore_index=True)

C:\Users\the_s\anaconda3\lib\site-packages\numpy\lib\function_base.py:2683: RuntimeWarning: Degrees of freedom <= 0 for slice
  c = cov(x, y, rowvar, dtype=dtype)
C:\Users\the_s\anaconda3\lib\site-packages\numpy\lib\function_base.py:2542: RuntimeWarning: divide by zero encountered in true_divide
  c *= np.true_divide(1, fact)
C:\Users\the_s\anaconda3\lib\site-packages\numpy\lib\function_base.py:2542: RuntimeWarning: invalid value encountered in multiply
  c *= np.true_divide(1, fact)
C:\Users\the_s\anaconda3\lib\site-packages\numpy\lib\function_base.py:2683: RuntimeWarning: Degrees of freedom <= 0 for slice
  c = cov(x, y, rowvar, dtype=dtype)
C:\Users\the_s\anaconda3\lib\site-packages\numpy\lib\function_base.py:2542: RuntimeWarning: divide by zero encountered in true_divide
  c *= np.true_divide(1, fact)
C:\Users\the_s\anaconda3\lib\site-packages\numpy\lib\function_base.py:2542: RuntimeWarning: invalid value encountered in multiply
  c *= np.true_divide(1, fact)
C:\Users\the_s\anacond

In [7]:
pc_df_2['image'] = pc_df_2['image'].astype(float)
pc_df_2 = pc_df_2.sort_values(['tto','image'])
pc_df_2.to_csv("pc_cells.csv")